Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 10 rows, 12 columns and 29 nonzeros
Model fingerprint: 0x72c66c83
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 20.0000000
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 8 rows, 10 columns, 27 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: cutoff, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        20.00000   20.00000  0.00%     -   

In [6]:
import pandas as pd 
import numpy as np 
from graphStructure import graph 
from structure import node
import gurobipy as gp
from gurobipy import GRB 
from gurobipy import quicksum
import random

def subtour_elimination(model, x, n):
    for i in range(n):
        for j in range(i+1, n):
            for k in range(j+1, n):
                model.addConstr(x[i,j] + x[j,k] + x[k,i] <= 2)

n = 4 # Number of nodes

# Create a new model
model = gp.Model("TSP")

#Read data of sensors
sensorData = pd.read_excel('Data/Environement/K=12/SensorInformation.xls')
sensorCoordinate = []

#Sensors coordinate 
for row in sensorData.index:
    coordinate = (sensorData['X'][row] ,sensorData['Y'][row] ,sensorData['Cluster_Index'][row])
    sensorCoordinate.append(coordinate)
sensorInformations = sensorCoordinate 

#Cluster Information 
clusterData = pd.read_excel('Data/Environement/K=12/ClustersInformations.xls')
ClusterInformation = []
AllClusterIndex = []

#Cluster coordinate 
for row in clusterData.index:
    coordinate = (clusterData['X'][row] ,clusterData['Y'][row] , clusterData['Cluster_index'][row])
    ClusterInformation.append(coordinate)
    AllClusterIndex.append(clusterData['Cluster_index'][row])
C= AllClusterIndex[0:-1]
#Create a graph structure 
predecessor = []
sucessor =[]
Nodesss = []
for i in range(len(AllClusterIndex)):
    predecessor = []
    sucessor =[]
    for j in range(len(AllClusterIndex)):
        if i==len(AllClusterIndex)-1 and i!=j:
            sucessor.append(j)
        if i!=j and j != len(AllClusterIndex)-1 and i!= len(AllClusterIndex)-1 :
            predecessor.append(j)
            sucessor.append(j)
    g = graph(i , predecessor , sucessor)
    Nodesss.append(g)
N = node(Nodesss)
AllClusterIndex

# Normlized Data of the Mission 
nomlizedMissionData = pd.read_excel("NormlizedData.xls")
missionInformationSubPath = []
missionInformationScores =  []
for row in nomlizedMissionData.index:
    subPath = (nomlizedMissionData['Cluster_ID'][row] , nomlizedMissionData['Cluster_DESTINATION'][row])
    scores  =(float(nomlizedMissionData['TIME_TRAVEL_CONSUMPTION'][row]),float(nomlizedMissionData['HOVERING_TIME'][row]) , float(nomlizedMissionData['ENERGY_HOVER'][row])  , float(nomlizedMissionData['TRAVEL_ENERGY_COST'][row]))
    missionInformationSubPath.append(subPath)
    missionInformationScores.append(scores)
#--------------------------------------------------------------------------

dataDict = {}
for cluster in range(len(missionInformationSubPath)):
    dataDict.update({missionInformationSubPath[cluster] : missionInformationScores[cluster]})

#Match subpath with there scores  
combinations , timeTravel , hoveringTime , energyTravel , hoveringEnergy  = gp.multidict(dataDict)

# Create binary variables x[i,j] indicating if edge (i,j) is in the tour
x = model.addVars(combinations, vtype=GRB.BINARY, name="x")
TimeTravel = model.addVars(timeTravel , vtype = GRB.BINARY ,name = 'timeTravel')
HoveringTime =  model.addVars(hoveringTime , vtype = GRB.BINARY ,name = 'hoveringTime')
EnergyTravel =  model.addVars(energyTravel , vtype = GRB.BINARY ,name = 'energyTravel')
HoveringEnergy = model.addVars(hoveringEnergy , vtype = GRB.BINARY ,name = 'hoveringEnergy')
#---------------------------------------------------------------------
#The startingPosition Should be the base station  (C1) --> keep  
startingPosition = 3
numberOfCluster = 4
#----------------------------------------------------------------------
# Limit the number of edges to be less than or equal to the number of nodes minus one
model.addConstrs((x.sum(i, '*') <= n-1 for i in range(startingPosition)))
for i in range(numberOfCluster):
    model.addConstr(quicksum(x[i,j] for j in range(numberOfCluster) if i!=j and j!= startingPosition ) == 1 , name="out_%d" % i)
    model.addConstr(quicksum(x[j,i] for j in range(numberOfCluster) if i!=j and i!= startingPosition ) == 1, name="in_%d" % i)
#------------------------------------------------------------------------

# Implement subtour elimination constraints
subtour_elimination(model, x, startingPosition)
#------------------------------------------------------------------------

# Set the objective to minimize the total distance
λ1  = random.uniform(0,1)
λ2  = 1 - λ1  
model.NumObj = 2 
model.modelSense = GRB.MINIMIZE 
model.setObjectiveN(quicksum(TimeTravel[i,j] + HoveringTime[i,j] for  i,j in x) , index=0 , weight = λ1)
model.setObjectiveN(quicksum(EnergyTravel[i,j] + HoveringEnergy[i,j] for i,j in x) , index=1 , weight = λ2 )
#--------------------------------------------------------------------------

# Solve the model
model.optimize()

# Print the solution
# print("Optimal tour:")
# for i in range(n):
#     for j in range(i+1, startingPosition):
#         if x[i,j].x > 0.5:
#             print(f"{i} -> {j}")


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 12 rows, 45 columns and 27 nonzeros
Model fingerprint: 0x8aeec376
Variable types: 0 continuous, 45 integer (45 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 12 rows, 45 columns and 27 nonzeros
Model fingerprint: 0x62005570
Variable types: 0 continuous, 45 integer (

In [ ]:
from gurobipy import Model, GRB, quicksum



def add_subtour_elimination_constraints(model, n, x):
    # Create an empty list to store the subtour elimination constraints
    subtour_constraints = []
    for i in range(1, n):
        for j in range(i+1, n):
            subtour_constraints.append(model.addConstr(
                quicksum(x[i,k] for k in range(n) if k != i) + 
                quicksum(x[k,j] for k in range(n) if k != j) <= 
                quicksum(x[i,j] for i in range(n)) - 1, "subtour_" + str(i) + "_" + str(j)))
    return subtour_constraints
# Initialize the model and variables
model = Model("UAV ENERGY MINIMIZATION")

# Add subtour elimination constraints
subtour_constraints = add_subtour_elimination_constraints(model, n, x)
model.optimize()
